In [ ]:
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta
import os
import subprocess
import requests
import json

# Define default arguments
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Function to fork a GitHub repository
def fork_github_repo(username, github_token, template_repo_owner, template_repo_name, github_org=None, **kwargs):
    headers = {
        'Authorization': f'token {github_token}',
        'Accept': 'application/vnd.github.v3+json'
    }
    
    # New repository name
    new_repo_name = f"{username}-hugo-site"
    
    # API endpoint for forking
    fork_url = f'https://api.github.com/repos/{template_repo_owner}/{template_repo_name}/forks'
    
    # Fork parameters
    fork_data = {}
    if github_org:
        fork_data['organization'] = github_org
    
    # Create the fork
    response = requests.post(fork_url, headers=headers, data=json.dumps(fork_data))
    
    if response.status_code not in [202, 200]:
        raise Exception(f"Failed to fork repository: {response.text}")
    
    fork_info = response.json()
    
    # If we need to rename the forked repository
    if github_org:
        owner = github_org
    else:
        owner = fork_info['owner']['login']
    
    # Rename the forked repository if needed
    if fork_info['name'] != new_repo_name:
        rename_url = f'https://api.github.com/repos/{owner}/{fork_info["name"]}'
        rename_data = {'name': new_repo_name}
        
        rename_response = requests.patch(rename_url, headers=headers, data=json.dumps(rename_data))
        
        if rename_response.status_code != 200:
            raise Exception(f"Failed to rename repository: {rename_response.text}")
        
        repo_info = rename_response.json()
    else:
        repo_info = fork_info
    
    return {
        'repo_name': new_repo_name,
        'clone_url': repo_info['clone_url'],
        'html_url': repo_info['html_url'],
        'owner': owner
    }

# Function to update the Hugo configuration for user
def update_hugo_config(username, repo_info, github_token, **kwargs):
    repo_name = repo_info['repo_name']
    clone_url = repo_info['clone_url']
    owner = repo_info['owner']
    base_dir = f"/tmp/{repo_name}"
    
    # Create directory
    os.makedirs(base_dir, exist_ok=True)
    os.chdir(base_dir)
    
    # Clone the repository
    auth_url = clone_url.replace('https://', f'https://oauth2:{github_token}@')
    subprocess.run(["git", "clone", auth_url, "."])
    
    # Update configuration for this user
    with open("config.toml", "r") as f:
        config = f.read()
    
    # Update deployment URL for this user
    config = config.replace(
        'URL = "gs://test-02apr/template"', 
        f'URL = "gs://test-02apr/{username}"'
    )
    
    # Update site title or other user-specific configuration
    config = config.replace(
        'title = "Hugo Template Site"',
        f'title = "{username}\'s Hugo Site"'
    )
    
    with open("config.toml", "w") as f:
        f.write(config)
    
    # Update GitHub Actions workflow if needed
    workflow_path = ".github/workflows/deploy.yml"
    if os.path.exists(workflow_path):
        with open(workflow_path, "r") as f:
            workflow = f.read()
        
        # Update deployment path in workflow
        workflow = workflow.replace(
            'gs://test-02apr/template',
            f'gs://test-02apr/{username}'
        )
        
        with open(workflow_path, "w") as f:
            f.write(workflow)
    
    # Commit and push changes
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", f"Update configuration for {username}"])
    subprocess.run(["git", "push", "origin", "main"])
    
    # Build the site locally (for GCS deployment)
    subprocess.run(["hugo", "--minify"])
    
    return base_dir

# Function to set GitHub repository secrets (for GCP credentials)
def set_github_secrets(repo_info, github_token, secrets, **kwargs):
    headers = {
        'Authorization': f'token {github_token}',
        'Accept': 'application/vnd.github.v3+json'
    }
    
    repo_name = repo_info['repo_name']
    owner = repo_info['owner']
    
    # Get the public key for the repository to encrypt secrets
    key_response = requests.get(
        f'https://api.github.com/repos/{owner}/{repo_name}/actions/secrets/public-key',
        headers=headers
    )
    
    if key_response.status_code != 200:
        raise Exception(f"Failed to get public key: {key_response.text}")
    
    key_data = key_response.json()
    public_key = key_data['key']
    key_id = key_data['key_id']
    
    # Add each secret
    for secret_name, secret_value in secrets.items():
        # In a real implementation, you would encrypt the secret value with the public key
        # This is a placeholder - encrypt properly in production
        encrypted_secret = secret_value  # This should be encrypted
        
        secret_data = {
            'encrypted_value': encrypted_secret,
            'key_id': key_id
        }
        
        secret_response = requests.put(
            f'https://api.github.com/repos/{owner}/{repo_name}/actions/secrets/{secret_name}',
            headers=headers,
            data=json.dumps(secret_data)
        )
        
        if secret_response.status_code not in [201, 204]:
            raise Exception(f"Failed to set secret {secret_name}: {secret_response.text}")

# Function to deploy to GCS
def deploy_to_gcs(username, base_dir, **kwargs):
    # Build path
    public_dir = os.path.join(base_dir, "public")
    
    # Deploy to GCS
    subprocess.run([
        "gsutil", "-m", "rsync", "-r", 
        public_dir, 
        f"gs://test-02apr/{username}"
    ])
    
    return f"https://storage.googleapis.com/test-02apr/{username}/index.html"

# Create DAG
with DAG(
    'hugo_site_fork_template',
    default_args=default_args,
    description='Create user Hugo site by forking template repository',
    schedule_interval=None,  # Only triggered manually
    start_date=datetime(2023, 1, 1),
    catchup=False,
) as dag:
    
    # Define parameters as DAG parameters
    username = '{{ dag_run.conf["username"] }}'
    github_token = '{{ var.value.github_token }}'  # Store this as an Airflow variable
    template_repo_owner = '{{ var.value.template_repo_owner }}'  # Owner of the template repo
    template_repo_name = '{{ var.value.template_repo_name }}'  # Name of the template repo
    github_org = '{{ var.value.github_org }}'  # Optional: GitHub organization name
    gcp_project_id = '{{ var.value.gcp_project_id }}'
    gcp_sa_key = '{{ var.value.gcp_sa_key }}'  # Store this as an Airflow variable
    
    # Task to fork GitHub repository
    fork_repo = PythonOperator(
        task_id='fork_github_repo',
        python_callable=fork_github_repo,
        op_kwargs={
            'username': username,
            'github_token': github_token,
            'template_repo_owner': template_repo_owner,
            'template_repo_name': template_repo_name,
            'github_org': github_org if github_org != '' else None
        },
    )
    
    # Task to update Hugo configuration
    update_config = PythonOperator(
        task_id='update_hugo_config',
        python_callable=update_hugo_config,
        op_kwargs={
            'username': username,
            'repo_info': '{{ task_instance.xcom_pull(task_ids="fork_github_repo") }}',
            'github_token': github_token
        },
    )
    
    # Task to set GitHub secrets
    set_secrets = PythonOperator(
        task_id='set_github_secrets',
        python_callable=set_github_secrets,
        op_kwargs={
            'repo_info': '{{ task_instance.xcom_pull(task_ids="fork_github_repo") }}',
            'github_token': github_token,
            'secrets': {
                'GCP_PROJECT_ID': gcp_project_id,
                'GCP_SA_KEY': gcp_sa_key
            }
        },
    )
    
    # Task to deploy to GCS
    deploy_site = PythonOperator(
        task_id='deploy_to_gcs',
        python_callable=deploy_to_gcs,
        op_kwargs={
            'username': username,
            'base_dir': '{{ task_instance.xcom_pull(task_ids="update_hugo_config") }}',
        },
    )
    
    # Task to clean up
    cleanup = BashOperator(
        task_id='cleanup',
        bash_command='rm -rf {{ task_instance.xcom_pull(task_ids="update_hugo_config") }}',
    )
    
    # Set task dependencies
    fork_repo >> update_config >> set_secrets >> deploy_site >> cleanup

In [ ]:
# old

In [ ]:
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta
import os
import subprocess
import requests
import json

# Define default arguments
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Function to create GitHub repository
def create_github_repo(username, github_token, github_org=None, **kwargs):
    headers = {
        'Authorization': f'token {github_token}',
        'Accept': 'application/vnd.github.v3+json'
    }
    
    repo_name = f"{username}-hugo-site"
    
    # Set up the repository data
    repo_data = {
        'name': repo_name,
        'description': f'Hugo website for {username}',
        'private': True,  # Set to False if you want public repositories
        'auto_init': True  # Initialize with README
    }
    
    # Create in organization or personal account
    if github_org:
        url = f'https://api.github.com/orgs/{github_org}/repos'
    else:
        url = 'https://api.github.com/user/repos'
    
    # Create the repository
    response = requests.post(url, headers=headers, data=json.dumps(repo_data))
    
    if response.status_code not in [201, 200]:
        raise Exception(f"Failed to create repository: {response.text}")
    
    repo_info = response.json()
    return {
        'repo_name': repo_name,
        'clone_url': repo_info['clone_url'],
        'html_url': repo_info['html_url']
    }

# Function to create Hugo site and push to GitHub
def create_and_push_hugo_site(username, repo_info, github_token, **kwargs):
    repo_name = repo_info['repo_name']
    clone_url = repo_info['clone_url']
    base_dir = f"/tmp/{repo_name}"
    
    # Create directory
    os.makedirs(base_dir, exist_ok=True)
    os.chdir(base_dir)
    
    # Clone the repository (it should have a README already)
    # Modify the clone URL to include the token for authentication
    auth_url = clone_url.replace('https://', f'https://oauth2:{github_token}@')
    subprocess.run(["git", "clone", auth_url, "."])
    
    # Initialize Hugo site
    subprocess.run(["hugo", "new", "site", ".", "--force"])
    
    # Add theme as submodule
    subprocess.run([
        "git", "submodule", "add", 
        "https://github.com/theNewDynamic/gohugo-theme-ananke", 
        "themes/ananke"
    ])
    
    # Add theme config
    with open("config.toml", "a") as f:
        f.write('theme = "ananke"\n')
        f.write('\n[deployment]\n')
        f.write('  [[deployment.targets]]\n')
        f.write('    name = "gcs_hosting"\n')
        f.write(f'    URL = "gs://test-02apr/{username}"\n')
    
    # Create GitHub Actions workflow
    os.makedirs(".github/workflows", exist_ok=True)
    with open(".github/workflows/deploy.yml", "w") as f:
        f.write("""name: Deploy Hugo site to GCS

on:
  push:
    branches:
      - main

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
        with:
          submodules: true

      - name: Setup Hugo
        uses: peaceiris/actions-hugo@v2
        with:
          hugo-version: 'latest'
          extended: true

      - name: Build
        run: hugo --minify

      - name: Setup Cloud SDK
        uses: google-github-actions/setup-gcloud@v1
        with:
          project_id: ${{ secrets.GCP_PROJECT_ID }}
          service_account_key: ${{ secrets.GCP_SA_KEY }}

      - name: Deploy
        run: gsutil -m rsync -r ./public gs://test-02apr/""" + username)
    
    # Create content
    subprocess.run(["hugo", "new", "posts/welcome.md"])
    
    # Commit and push all changes
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", "Initial Hugo site setup"])
    subprocess.run(["git", "push", "origin", "main"])
    
    # Build the site locally (for GCS deployment)
    subprocess.run(["hugo", "--minify"])
    
    return base_dir

# Function to set GitHub repository secrets (for GCP credentials)
def set_github_secrets(repo_info, github_token, secrets, github_org=None, **kwargs):
    headers = {
        'Authorization': f'token {github_token}',
        'Accept': 'application/vnd.github.v3+json'
    }
    
    repo_name = repo_info['repo_name']
    owner = github_org if github_org else kwargs['github_username']
    
    # Get the public key for the repository to encrypt secrets
    key_response = requests.get(
        f'https://api.github.com/repos/{owner}/{repo_name}/actions/secrets/public-key',
        headers=headers
    )
    
    if key_response.status_code != 200:
        raise Exception(f"Failed to get public key: {key_response.text}")
    
    key_data = key_response.json()
    public_key = key_data['key']
    key_id = key_data['key_id']
    
    # Add each secret
    for secret_name, secret_value in secrets.items():
        # In a real implementation, you would encrypt the secret value with the public key
        # For simplicity, we're skipping the actual encryption step here
        # You would need to use the sodium library for proper encryption
        
        # This is a placeholder - in a real implementation, encrypt the secret properly
        encrypted_secret = secret_value  # This should be encrypted
        
        secret_data = {
            'encrypted_value': encrypted_secret,
            'key_id': key_id
        }
        
        secret_response = requests.put(
            f'https://api.github.com/repos/{owner}/{repo_name}/actions/secrets/{secret_name}',
            headers=headers,
            data=json.dumps(secret_data)
        )
        
        if secret_response.status_code not in [201, 204]:
            raise Exception(f"Failed to set secret {secret_name}: {secret_response.text}")

# Function to deploy to GCS
def deploy_to_gcs(username, base_dir, **kwargs):
    # Build path
    public_dir = os.path.join(base_dir, "public")
    
    # Deploy to GCS
    subprocess.run([
        "gsutil", "-m", "rsync", "-r", 
        public_dir, 
        f"gs://test-02apr/{username}"
    ])
    
    return f"https://storage.googleapis.com/test-02apr/{username}/index.html"

# Create DAG
with DAG(
    'hugo_site_creation_with_github',
    default_args=default_args,
    description='Create and deploy Hugo site for a user with GitHub repo',
    schedule_interval=None,  # Only triggered manually
    start_date=datetime(2023, 1, 1),
    catchup=False,
) as dag:
    
    # Define parameters as DAG parameters
    username = '{{ dag_run.conf["username"] }}'
    github_token = '{{ var.value.github_token }}'  # Store this as an Airflow variable
    github_username = '{{ var.value.github_username }}'  # Your GitHub username or organization
    gcp_project_id = '{{ var.value.gcp_project_id }}'
    gcp_sa_key = '{{ var.value.gcp_sa_key }}'  # Store this as an Airflow variable
    
    # Task to create GitHub repository
    create_repo = PythonOperator(
        task_id='create_github_repo',
        python_callable=create_github_repo,
        op_kwargs={
            'username': username,
            'github_token': github_token,
            'github_username': github_username
        },
    )
    
    # Task to create Hugo site and push to GitHub
    create_site = PythonOperator(
        task_id='create_and_push_hugo_site',
        python_callable=create_and_push_hugo_site,
        op_kwargs={
            'username': username,
            'repo_info': '{{ task_instance.xcom_pull(task_ids="create_github_repo") }}',
            'github_token': github_token
        },
    )
    
    # Task to set GitHub secrets
    set_secrets = PythonOperator(
        task_id='set_github_secrets',
        python_callable=set_github_secrets,
        op_kwargs={
            'repo_info': '{{ task_instance.xcom_pull(task_ids="create_github_repo") }}',
            'github_token': github_token,
            'github_username': github_username,
            'secrets': {
                'GCP_PROJECT_ID': gcp_project_id,
                'GCP_SA_KEY': gcp_sa_key
            }
        },
    )
    
    # Task to deploy to GCS
    deploy_site = PythonOperator(
        task_id='deploy_to_gcs',
        python_callable=deploy_to_gcs,
        op_kwargs={
            'username': username,
            'base_dir': '{{ task_instance.xcom_pull(task_ids="create_and_push_hugo_site") }}',
        },
    )
    
    # Task to clean up
    cleanup = BashOperator(
        task_id='cleanup',
        bash_command='rm -rf {{ task_instance.xcom_pull(task_ids="create_and_push_hugo_site") }}',
    )
    
    # Set task dependencies
    create_repo >> create_site >> set_secrets >> deploy_site >> cleanup